# Jeffray Tsai Final Project
## Option A: Functional/API Project

In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [32]:
import requests 
import os
import json
import pandas as pd
import pytest
import pandas as pd
from datetime import datetime

In [10]:
#Token refresher
requests.get('https://oauth.reddit.com/api/v1/me.json', headers=headers)

<Response [200]>

In [11]:
#Search through Wall Street Bets subreddit
r = requests.get('https://www.reddit.com/r/wallstreetbets/hot.json',
                headers=headers)

In [12]:
r.status_code

200

In [13]:
print(json.dumps(r.json(), indent=2,sort_keys=True))

{
  "data": {
    "after": "t3_rhg0r2",
    "before": null,
    "children": [
      {
        "data": {
          "all_awardings": [
            {
              "award_sub_type": "GLOBAL",
              "award_type": "global",
              "awardings_required_to_grant_benefits": null,
              "coin_price": 150,
              "coin_reward": 0,
              "count": 1,
              "days_of_drip_extension": 0,
              "days_of_premium": 0,
              "description": "Thank you stranger. Shows the award.",
              "end_date": null,
              "giver_coin_reward": null,
              "icon_format": null,
              "icon_height": 2048,
              "icon_url": "https://i.redd.it/award_images/t5_22cerq/klvxk1wggfd41_Helpful.png",
              "icon_width": 2048,
              "id": "award_f44611f1-b89e-46dc-97fe-892280b13b82",
              "is_enabled": true,
              "is_new": false,
              "name": "Helpful",
              "penny_donate": null,
 

In [14]:
#Use JSON
json_response = r.json()

In [15]:
#Make dataframe
df = pd.DataFrame()

In [16]:
#Organize dataframe 
for post in r.json()['data']['children']:
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score'],
        'created_date': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%d'),
    },ignore_index=True)

In [17]:
#Show first 5 rows of dataframe
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,created_date
0,wallstreetbets,"What Are Your Moves Tomorrow, December 17, 2021",What are your moves tomorrow? Please keep the ...,0.94,82.0,0.0,82.0,2021-12-16
1,wallstreetbets,Most Anticipated Earnings Releases for the tra...,,0.98,844.0,0.0,844.0,2021-12-11
2,wallstreetbets,My ancestor 50 000 BCE.,,0.95,36425.0,0.0,36425.0,2021-12-16
3,wallstreetbets,Always inverse Jim Cramer,,0.97,3652.0,0.0,3652.0,2021-12-16
4,wallstreetbets,title,,0.98,1143.0,0.0,1143.0,2021-12-16


In [39]:
df.to_csv('posts.csv')

In [18]:
#Make a function for getting subreddit dataframe 
def get_results(limit_only,token):
    """
    This function to obtain posts from WallStreetBets subreddit. 
    Parameters: limit_only - the maximum number of items to return. 
    Typical example: 
    |subreddit      |title                     |selftext             |upvote_ratio|ups  |downs|score|created_utc
    |wallstreetbets |Daily Discussion Thread...|Your daily trading...|0.90        |245.0|0.0  |245.0|2021-12-16T06:00:12Z
    """
    requests.get('https://oauth.reddit.com/api/v1/me.json', headers=headers)
    if limit_only == limit_only:
        limit = limit_only
    response = requests.get('https://www.reddit.com/r/wallstreetbets/hot.json',
                headers=headers,params={'limit':limit})
    print(response)
    df = pd.DataFrame()
    for post in r.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'created_date': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%d'),
        },ignore_index=True)
    return df

In [35]:
def post_search(keyword, df):
    """
    This function is primarily used to search for keywords within posts. 
    Parameters: keyword - the function will search for the keyword in the title and post's text, df - the dataframe used.
    Typical example:
    |subreddit     |title                         |selftext                         |upvote_ratio|ups  |downs|score|created_utc
    |wallstreetbets|Are you buying the Reddit ipo?|Are you buying the Reddit ipo?...|0.91        |903.0|0.0  |903.0|2021-12-16T08:48:49Z
    """
    search = '|'.join(keyword)
    df = df[(df['title'].str.lower().str.contains(keyword,na=False))|(df['selftext'].str.lower().str.contains(keyword,na=False))]
    return df

In [36]:
#Save as post variable for testing
post = post_search('invest',df)
post

,subreddit,title,selftext,upvote_ratio,ups,downs,score,created_date
17,wallstreetbets,I wish I invested more but happy with the returns,,0.93,321.0,0.0,321.0,2021-12-16
19,wallstreetbets,PLTR CEO wishes all investors Happy Holidays a...,,0.93,554.0,0.0,554.0,2021-12-16


In [37]:
#Make a test function

def test_post_search():
    keyword = 'invest'
    expected = 17
    actual = post_search(keyword,df).index[0]
    assert actual == expected

In [38]:
#Test function worked 
test_post_search()

In [25]:
def upvotes_search(low, high, df):
    """
    This function will search for posts with a certain number of upvotes that are between the given parameter. 
    Parameters: low/high - the number of upvotes should between these two numbers, df - the dataframe used. 
    Typical example: 
    |subreddit     |title                     |selftext                               |upvote_ratio|ups  |downs|score|created_utc
    |wallstreetbets|Daily Discussion Thread...|Your daily trading discussion thread...|0.90        |245.0|0.0  |245.0|2021-12-16T06:0012Z
    """
    try:
        if low < 0: raise Error()
        if high < 0: raise Error()
        if high < low: raise Error()
        else:
            df = df[df['ups'].between(low,high)]
            return df
    except: 
        print('Please input different values.')
        pass

In [26]:
#Search for posts with 200 upvotes to 1,000 upvotes
upvotes_search(200,1000,df)

,subreddit,title,selftext,upvote_ratio,ups,downs,score,created_date
1,wallstreetbets,Most Anticipated Earnings Releases for the tra...,,0.98,844.0,0.0,844.0,2021-12-11
5,wallstreetbets,Reddit management hoping WSB plays nice with t...,,0.97,712.0,0.0,712.0,2021-12-16
6,wallstreetbets,"With the upcoming Reddit IPO, I feel it's impo...",,0.98,832.0,0.0,832.0,2021-12-16
9,wallstreetbets,"I think Bulls and Bears alike, need this right...",,0.98,409.0,0.0,409.0,2021-12-16
10,wallstreetbets,I laugh - But I also hold 400 T shares. Lord h...,,0.98,381.0,0.0,381.0,2021-12-16
11,wallstreetbets,It is time,,0.90,319.0,0.0,319.0,2021-12-16
13,wallstreetbets,From $414 to over $6k in two days! Thanks Pfiz...,,0.93,223.0,0.0,223.0,2021-12-16
17,wallstreetbets,I wish I invested more but happy with the returns,,0.93,321.0,0.0,321.0,2021-12-16
19,wallstreetbets,PLTR CEO wishes all investors Happy Holidays a...,,0.93,554.0,0.0,554.0,2021-12-16
20,wallstreetbets,"Daily Discussion Thread for December 16, 2021",Your daily trading discussion thread. Please k...,0.90,301.0,0.0,301.0,2021-12-16


In [27]:
#Try to get an error
upvotes_search(-1,2,df)

Please input different values.


In [28]:
def upvote_ratio(low,high,df):
    """
    This function will search for posts with a certain ratio of upvotes that are between the given parameter. 
    Parameters: low/high - the ratio should be between these two numbers, df - the dataframe used. 
    """
    try:
        if low < 0 or low > 1: raise Error()
        if high < 0 or high > 1: raise Error()
        if high < low: raise Error()
        else:
            df = df[df['upvote_ratio'].between(low,high)]
            return df
    except: 
        print('Please input different values.')
        pass

In [29]:
#Search for posts with an upvote ratio between 0.91 and 0.92
upvote_ratio(0.91,0.92,df)

,subreddit,title,selftext,upvote_ratio,ups,downs,score,created_date
7,wallstreetbets,Are you buying the Reddit ipo?,Are you buying the Reddit ipo?\n\nOr do you th...,0.92,1430.0,0.0,1430.0,2021-12-16
8,wallstreetbets,Let’s not forget the day President Kennedy spo...,,0.91,36599.0,0.0,36599.0,2021-12-15


In [30]:
#Try to get an error
upvote_ratio(0.92,0.91,df)

Please input different values.
